# Transform

Transfom represent kinds of transform matrixes, including translation, scaling, rotation and projection.

## Identity

In [ ]:
from py3d import Transform
Transform()


In [ ]:
assert _.sum() == 4


In [ ]:
Transform(n=(2,))


In [ ]:
assert _.sum() == 8


## Translation

### Translation Matrix

In **py3d**, transformations are represented as left-multiplication matrixes, point are represented as row vectors.

#### Left-multiplication translation matrix

In [ ]:
from sympy import symbols, Matrix
x, y, z, dx, dy, dz = symbols("x y z dx dy dz")
point = Matrix([x, y, z, 1]).T
translation = Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [dx, dy, dz, 1]
])
translation


In [ ]:
point * translation


In [ ]:
dx_,dy_,dz_ = symbols("dx' dy' dz'")
translation_ = Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [dx_, dy_, dz_, 1]
])
translation_ * translation

In [ ]:
translation * translation_

#### Right-multiplication translation matrix

In [ ]:
translation.T


In [ ]:
translation.T * point.T


In [ ]:
from py3d import Vector3
Vector3([2, 3, 4]).as_translation()


In [ ]:
import numpy
assert numpy.array_equal(_[3, 0:3], [2, 3, 4])


In [ ]:
Transform.from_translation(x=1, n=(2,))


Translate a series of points

In [ ]:
import py3d
v = py3d.Viewer()
points = py3d.Vector3.Rand(100)
v.render(points.as_point())
points @= py3d.Transform.from_translation(x=1)
v.render(points.as_point())
v.show()


Move a car

In [ ]:
import py3d
v = py3d.Viewer()
car = py3d.Utils.Car()
grid = py3d.Utils.Grid(10)
t = 0
while t < 4:
    car.vertex @= py3d.Transform.from_translation(x=0.2)
    v.render(grid, car, t=t)
    t += 0.1
v.show()


## Scaling

In [ ]:
from py3d import Vector3
Vector3(x=1, y=2, z=(1, 2, 3)).as_scaling()


In [ ]:
Transform.from_scaling(x=1, y=-1, z=(1, 2, 3))


In [ ]:
from py3d import Vector3, Transform
points = (Vector3.Rand(1000)-0.5).U @ Transform.from_scaling(x=2)
points.as_point().render()


## [Rotation](Rotation.html)

## Projection

### Camera and scene

In [ ]:
import py3d
v=py3d.Viewer()
v.render(py3d.Utils.Grid())
camera=py3d.Utils.Axis(2)
camera.vertex @= py3d.Transform.from_translation([0,0,5]) @ py3d.Transform.from_rpy([py3d.pi/3,py3d.pi/6,0])
line=camera.vertex[0,0].as_vector()
v.render(camera,line)
v.show()

In [ ]:
camera.vertex[0][0]

### Project 3d points on xy plane

In [ ]:
import py3d
v=py3d.Viewer()
p=(py3d.Vector3.Rand(1000)-0.5).U+py3d.Vector3(x=1,y=1,z=2)
v.render(p.as_point())
p.z=0
v.render(p.as_point(), py3d.Utils.Grid())
v.show()

### Orthographic projection

In [ ]:
import sympy
l, r, t, b, n, f = sympy.symbols("l r t b n f")
m_scale = sympy.Matrix([
    [2/(r-l), 0, 0, 0],
    [0, 2/(t-b), 0, 0],
    [0, 0, 2/(n-f), 0],
    [0, 0, 0, 1]
])
m_translate = sympy.Matrix([
    [1, 0, 0, -(l+r)/2],
    [0, 1, 0, -(b+t)/2],
    [0, 0, 1, -(f+n)/2],
    [0, 0, 0, 1]
])
o=sympy.simplify(m_scale*m_translate).transpose()
o


In [ ]:
w, h = sympy.symbols("w h")
o=o.subs(l, -w/2).subs(r, w/2).subs(t, h/2).subs(b, -h/2)
o

Use perspective fov and aspect to define a orthographic projection

In [ ]:
fov, aspect, distance = sympy.symbols("fov a d")
o=o.subs(w, aspect * h).subs(h, distance*sympy.tan(fov/2)*2)
o

In [ ]:
x, y, z = sympy.symbols("x y z")
sympy.Matrix([x, y, z, 1]).T @ o

### Perspective projection

In [ ]:
from py3d import Transform, pi, Vector3
projection = Transform.from_perspective(pi/2, 1.2, 0, 1000)
projection


In [ ]:
from py3d import Camera, pi, Vector3, Transform
import matplotlib.pyplot as plt
camera = Camera()
camera.transform.translation = Transform.from_translation([0, 0, 10])
camera.transform.rotation = Transform.from_rpy([0, 0, 0])
camera.projection = Transform.from_perspective(pi/3, 1, 1, 1000)
points = Vector3.Rand(100)
proj = points @ camera.matrix
plt.gca().set_aspect(1)
plt.scatter(proj.x, proj.y)


In [ ]:
from numpy import allclose
from py3d import Transform, Vector3
assert (Transform([
    [1., 0., 0.3, 0.],
    [0., 1., 0., 0.],
    [0., 0., 0.1, 0.],
    [-1., 0., 0., 1.]
]) == [
    [1., 0., 0.3, 0.],
    [0., 1., 0., 0.],
    [0., 0., 0.1, 0.],
    [-1., 0., 0., 1.]
]).all()
assert Transform(n=(2, 3)).shape == (2, 3, 4, 4)
assert Transform(n=(4, 5)).n == (4, 5)
assert ((Vector3(x=1).H @ Transform())[..., 0:3] == Vector3(x=1)).all()
assert (Vector3(x=1) @ Transform() == Vector3(x=1)).all()
p = Vector3([1, 2, 3])
T = Transform.from_translation([1, 2, 3])
R = Transform.from_euler("xyz", [0, 0, 1])
assert allclose(p @ T @ R, p @ (T@R))


[↑Top](#Transform)

[←Home](index.html) 